# TO DO
1. First we need to read in the NFT to find the miner id parameters
2. Next we need to get the current ratio to ERG for the tokens to swap
3. Per ratio in the NFT itself we determine how much we are to end up sending to the given user
4. Build TX for ergo factory


In [1]:
from src.find_miner_id import ReadTokens

In [2]:
reader = ReadTokens()

In [5]:
wallet = '9ehJZvPDgvCNNd2zTQHxnSpcCAtb1kHbEN1VAgeoRD5DPVApYkk'
token = reader.get_latest_miner_id(wallet)
miner_id = reader.get_token_description(token['tokenId'])
miner_id

{'address': '9ehJZvPDgvCNNd2zTQHxnSpcCAtb1kHbEN1VAgeoRD5DPVApYkk',
 'height': 1299322,
 'minimumPayout': 33,
 'tokens': [{'token': 'ERGO', 'value': 50},
  {'token': 'SigUSD', 'value': 10},
  {'token': 'RSN', 'value': 10},
  {'token': 'rsBTC', 'value': 10},
  {'token': 'Gluon', 'value': 10},
  {'token': 'COMET', 'value': 5},
  {'token': 'Bober', 'value': 5}],
 'season': 0,
 'type': 'Miner ID',
 'fan_club': 'QX'}

In [7]:
tokens_to_swap = [item['token'] for item in miner_id['tokens']]
tokens_to_swap

['ERGO', 'SigUSD', 'RSN', 'rsBTC', 'Gluon', 'COMET', 'Bober']

In [16]:
swap = {'ERGO': 1,
         'SigUSD': 0.88,
         'RSN': 12.246,
         'rsBTC': 0.00001524,
         'Gluon': 1, # not on spectrum
         'COMET': 31141,
         'Bober': 3.875}

In [19]:
conversions = {}
payout = miner_id['minimumPayout']

for item in miner_id['tokens']:    
    token = item['token']
    ratio = item['value']
    ergs_to_swap = payout * ratio / 100
    token_conversion = swap[token]
    conversions[token] = token_conversion * ergs_to_swap

conversions

{'ERGO': 16.5,
 'SigUSD': 2.904,
 'RSN': 40.4118,
 'rsBTC': 5.0292e-05,
 'Gluon': 3.3,
 'COMET': 51382.649999999994,
 'Bober': 6.39375}

In [42]:
class ConvertTokens:
    def __init__(self):
        self.reader = ReadTokens()
        
    def get_miner_id(self, wallet):
        token = reader.get_latest_miner_id(wallet)
        miner_id = reader.get_token_description(token['tokenId'])
        return miner_id

    def get_token_conversions(self, wallet, miner_id):
        # need to read in price data from spectrum vs hard coding
        # swap = {'ERGO': 1,
        #      'SigUSD': 0.88,
        #      'RSN': 12.246,
        #      'rsBTC': 0.00001524,
        #      'Gluon': 1, # not on spectrum
        #      'COMET': 31141,
        #      'Bober': 3.875}
        spectrum_price_data = self.reader.get_api_data('https://api.spectrum.fi/v1/price-tracking/markets')
        tokens_to_swap = [item['token'] for item in miner_id['tokens']]
        
        my_spectrum_data = []
        swap = {}
        for data in spectrum_price_data:
            token = data['quoteSymbol']
            if data['baseSymbol'] == 'ERG' and token in tokens_to_swap:                
                swap[token] = data['lastPrice']
            if 'ERGO' in tokens_to_swap:
                swap['ERGO'] = 1
        return swap

    def create_conversion(self, miner_id, price_conversion):
        conversions = {}
        payout = miner_id['minimumPayout']
        
        for item in miner_id['tokens']:    
            token = item['token']
            ratio = item['value']
            ergs_to_swap = payout * ratio / 100
            try:
                token_conversion = price_conversion[token]
                conversions[token] = token_conversion * ergs_to_swap
            except KeyError:
                print('The TOKEN {} ISNT SUPPORTED on SPECTRUM'.format(token))
            
        
        return conversions

    def execute(self, wallet):
        miner_id = self.get_miner_id(wallet)
        price_conversion = self.get_token_conversions(wallet, miner_id)
        return self.create_conversion(miner_id, price_conversion)

converter = ConvertTokens()
results = converter.execute(wallet)
results

The TOKEN Gluon ISNT SUPPORTED on SPECTRUM


{'ERGO': 16.5,
 'SigUSD': 6.353466899999999,
 'RSN': 40.5319728,
 'rsBTC': 4.95e-05,
 'COMET': 1798.4998432499997,
 'Bober': 6.4154343}